In [1]:
import os
from pathlib import Path
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

In [3]:
pd.set_option('display.max_columns', 50)
pd.set_option('display.width', 120)

In [5]:
# read file
DATA_PATH = Path(r"D:/postgraduate/comp9417/group project/COMP9417_Bravo_Project/AirQualityUCI.csv")  # change to your own path

In [7]:
def load_airquality(path: Path) -> pd.DataFrame:
    if not path.exists():
        raise FileNotFoundError(f"File not found: {path}")
    if path.suffix.lower() in [".xlsx", ".xls"]:
        try:
            return pd.read_excel(path, sheet_name="AirQualityUCI")
        except Exception:
            return pd.read_excel(path)
    else:
        try:
            df = pd.read_csv(path, sep=';', decimal=',', low_memory=False)
        except Exception:
            df = pd.read_csv(path, low_memory=False)
        return df.dropna(axis=1, how='all')

df_raw = load_airquality(DATA_PATH)
print("Loaded shape:", df_raw.shape)
df_raw.head(3)

Loaded shape: (9471, 15)


,Date,Time,CO(GT),PT08.S1(CO),NMHC(GT),C6H6(GT),PT08.S2(NMHC),NOx(GT),PT08.S3(NOx),NO2(GT),PT08.S4(NO2),PT08.S5(O3),T,RH,AH
0,10/03/2004,18.00.00,2.6,1360.0,150.0,11.9,1046.0,166.0,1056.0,113.0,1692.0,1268.0,13.6,48.9,0.7578
1,10/03/2004,19.00.00,2.0,1292.0,112.0,9.4,955.0,103.0,1174.0,92.0,1559.0,972.0,13.3,47.7,0.7255
2,10/03/2004,20.00.00,2.2,1402.0,88.0,9.0,939.0,131.0,1140.0,114.0,1555.0,1074.0,11.9,54.0,0.7502


In [29]:
df = df_raw.copy()
if 'Date' in df.columns and 'Time' in df.columns:
    dt = pd.to_datetime(df['Date'].astype(str) + ' ' + df['Time'].astype(str), format= '%d/%m/%Y %H.%M.%S',
                        errors='coerce')
else:
    try:
        dt = pd.to_datetime(df.iloc[:, 0].astype(str), errors='coerce', dayfirst=True)
    except Exception:
        dt = pd.to_datetime(pd.Series([None]*len(df)), errors='coerce')

df = df.loc[dt.notna()].copy()
df.insert(0, 'Timestamp', dt[dt.notna()].values)
df.set_index('Timestamp', inplace=True)
df.sort_index(inplace=True)
print("After timestamp merge:", df.shape)

After timestamp merge: (9357, 15)


In [31]:
df.replace(-200, np.nan, inplace=True)  # -200 represent missing
for col in df.columns:
    if df[col].dtype == 'object':
        df[col] = pd.to_numeric(df[col].astype(str).str.replace(',', '.'), errors='coerce')

print("Missing (top 10):")
print(df.isna().sum().sort_values(ascending=False).head(10))

Missing (top 10):
Date             9357
Time             9357
NMHC(GT)         8443
CO(GT)           1683
NO2(GT)          1642
NOx(GT)          1639
PT08.S1(CO)       366
C6H6(GT)          366
PT08.S2(NMHC)     366
PT08.S3(NOx)      366
dtype: int64


In [33]:
df['hour']    = df.index.hour
df['weekday'] = df.index.weekday  # 0 stands for Mon
df['month']   = df.index.month
df[['hour','weekday','month']].head()

,hour,weekday,month
Timestamp,,,
2004-03-10 18:00:00,18,2,3
2004-03-10 19:00:00,19,2,3
2004-03-10 20:00:00,20,2,3
2004-03-10 21:00:00,21,2,3
2004-03-10 22:00:00,22,2,3


In [35]:
num_cols = df.select_dtypes(include='number').columns.tolist()
exclude = {'hour','weekday','month'}
to_scale = [c for c in num_cols if c not in exclude]

# exclude NaN and error rows
valid_cols = [c for c in to_scale if df[c].notna().sum() > 0 and np.isfinite(df[c]).all()]

scaler = StandardScaler()
df_scaled = df.copy()

for c in valid_cols:
    col_vals = df[[c]].dropna()
    if not col_vals.empty:
        scaled_vals = scaler.fit_transform(col_vals)
        df_scaled.loc[col_vals.index, f"{c}_z"] = scaled_vals

print("Scaled columns (first few):", valid_cols[:10], "...")
df_scaled.head(3)

Scaled columns (first few): [] ...


,Date,Time,CO(GT),PT08.S1(CO),NMHC(GT),C6H6(GT),PT08.S2(NMHC),NOx(GT),PT08.S3(NOx),NO2(GT),PT08.S4(NO2),PT08.S5(O3),T,RH,AH,hour,weekday,month
Timestamp,,,,,,,,,,,,,,,,,,
2004-03-10 18:00:00,NaN,NaN,2.6,1360.0,150.0,11.9,1046.0,166.0,1056.0,113.0,1692.0,1268.0,13.6,48.9,0.7578,18,2,3
2004-03-10 19:00:00,NaN,NaN,2.0,1292.0,112.0,9.4,955.0,103.0,1174.0,92.0,1559.0,972.0,13.3,47.7,0.7255,19,2,3
2004-03-10 20:00:00,NaN,NaN,2.2,1402.0,88.0,9.0,939.0,131.0,1140.0,114.0,1555.0,1074.0,11.9,54.0,0.7502,20,2,3


In [39]:
# pull result to local
out = Path("D:/postgraduate/comp9417/group project/COMP9417_Bravo_Project/"); out.mkdir(exist_ok=True, parents=True) #change to your local file path
df_scaled.to_csv(out / "AirQuality_preprocessed.csv", index=True)
#df_scaled.to_parquet(out / "AirQuality_preprocessed.parquet", index=True)
print("Saved to:", (out / "airquality_preprocessed.csv").resolve())

Saved to: D:\postgraduate\comp9417\group project\COMP9417_Bravo_Project\AirQuality_preprocessed.csv
